# Analyse data collected from the simulator

In [1]:
import csv
import cv2

lines = []
with open('simulator_images/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)
        
correction = 0.2
images = []
steering_angles = []
for line in lines:
    for i in range(3):
        source_path = line[i]
        filename = source_path.split('\\')[-1] #windows in local environment
        current_path = 'simulator_images/IMG/' + filename
        image = cv2.imread(current_path)
        images.append(image)
        if i == 1: #left image
            steering_angle = float(line[3]) + correction
            steering_angles.append(steering_angle)
        if i == 2: #right image
            steering_angle = float(line[3]) - correction
            steering_angles.append(steering_angle)
        if i == 0: #center image
            steering_angle = float(line[3])
            steering_angles.append(steering_angle)
        
    
augmented_images, augmented_steering_angles = [], []
for image, steering_angle in zip(images, steering_angles):
    augmented_images.append(image)
    augmented_steering_angles.append(steering_angle)
    augmented_images.append(cv2.flip(image, 1))
    augmented_steering_angles.append(steering_angle*-1.0)
    

    

# Alternative to load image as memory restrictions started to be visible

In [69]:
from sklearn.model_selection import train_test_split

import cv2
import numpy as np
import sklearn
from sklearn.utils import shuffle

import os
import csv

samples = []
with open('training_simulator_images/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)

correction = 0.2

train_samples, validation_samples = train_test_split(samples*6, test_size=0.2)

print("Train samples: " + str(len(train_samples)))
print("Validation samples: " + str(len(validation_samples)))

def generator(samples, batch_size=32):
    num_samples = len(samples)
    print(num_samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                name = 'training_simulator_images/IMG/'+batch_sample[0].split('\\')[-1]
                print(name)
                center_image = cv2.imread(name)
                center_angle = float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)
                #repeat process for left image
                left_name = 'training_simulator_images/IMG/'+batch_sample[1].split('\\')[-1]
                left_image = cv2.imread(left_name)
                left_angle = float(batch_sample[3]) + correction
                images.append(left_image)
                angles.append(left_angle)
                #repeat process for right image
                right_name = 'training_simulator_images/IMG/'+batch_sample[2].split('\\')[-1]
                right_image = cv2.imread(right_name)
                right_angle = float(batch_sample[3]) - correction
                images.append(right_image)
                angles.append(right_angle)
                
                #augmented images
                images.append(cv2.flip(center_image, 1))
                images.append(cv2.flip(left_image, 1))
                images.append(cv2.flip(right_image, 1))
                angles.append(center_angle*-1.0)
                angles.append(left_angle*-1.0)
                angles.append(right_angle*-1.0)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)
            
# Set our batch size
batch_size=6

print(len(train_samples))
# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=batch_size)
validation_generator = generator(validation_samples, batch_size=batch_size)



Train samples: 78614
Validation samples: 19654
78614


# Check size of images collected from training on Udacity Simulator

In [2]:
# this is no longer used

print(len(steering_angles))
print(len(images))

print(len(augmented_steering_angles))
print(len(augmented_images)) 

49134
49134
98268
98268


# Transform data into numpy arrays as Keras requires

In [5]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#first used images and angles, then augmented
X_train = np.array(augmented_images) # dataset for training
y_train = np.array(augmented_steering_angles) #labels, which in this case are the steering angles

MemoryError: Unable to allocate 14.1 GiB for an array with shape (98268, 160, 320, 3) and data type uint8

# Build a simple neural networks with keras just to validate that it works

In [70]:
from keras.models import Sequential
from keras.layers import Flatten, Dense
import math

model = Sequential()
model.add(Flatten(input_shape=(160, 320, 3)))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit_generator(train_generator, 
            steps_per_epoch=math.ceil(len(train_samples)/batch_size), 
            validation_data=validation_generator, 
            validation_steps=math.ceil(len(validation_samples)/batch_size), 
            epochs=5, verbose=1, shuffle=1)

model.save('model.h5')

Epoch 1/5
19654
training_simulator_images/IMG/center_2020_06_01_14_18_47_539.jpg
78614
training_simulator_images/IMG/center_2020_06_01_09_35_44_152.jpg
training_simulator_images/IMG/center_2020_06_01_13_22_42_849.jpg
training_simulator_images/IMG/center_2020_06_01_09_53_23_884.jpg
training_simulator_images/IMG/center_2020_06_01_09_45_00_828.jpg
training_simulator_images/IMG/center_2020_06_01_13_21_36_965.jpg
training_simulator_images/IMG/center_2020_06_01_09_35_04_358.jpg
training_simulator_images/IMG/center_2020_06_01_10_22_51_481.jpg
training_simulator_images/IMG/center_2020_06_01_14_15_08_217.jpg
training_simulator_images/IMG/center_2020_06_01_13_22_58_046.jpg
training_simulator_images/IMG/center_2020_06_01_09_47_18_158.jpg
training_simulator_images/IMG/center_2020_06_01_13_24_56_874.jpg
training_simulator_images/IMG/center_2020_06_01_14_19_11_807.jpg
training_simulator_images/IMG/center_2020_06_01_09_35_14_221.jpg
training_simulator_images/IMG/center_2020_06_01_14_18_09_245.jpg
tra

training_simulator_images/IMG/center_2020_06_01_09_47_36_977.jpg
training_simulator_images/IMG/center_2020_06_01_13_21_42_379.jpg
training_simulator_images/IMG/center_2020_06_01_09_37_13_227.jpg
training_simulator_images/IMG/center_2020_06_01_14_17_31_678.jpg
training_simulator_images/IMG/center_2020_06_01_09_47_38_896.jpg
training_simulator_images/IMG/center_2020_06_01_13_21_33_670.jpg
   10/13103 [..............................] - ETA: 30:56 - loss: 62166286.4869training_simulator_images/IMG/center_2020_06_01_13_23_34_609.jpg
training_simulator_images/IMG/center_2020_06_01_09_45_35_872.jpg
training_simulator_images/IMG/center_2020_06_01_09_54_25_694.jpg
training_simulator_images/IMG/center_2020_06_01_13_22_49_387.jpg
training_simulator_images/IMG/center_2020_06_01_13_22_40_008.jpg
training_simulator_images/IMG/center_2020_06_01_09_36_39_499.jpg
training_simulator_images/IMG/center_2020_06_01_14_16_19_602.jpg
training_simulator_images/IMG/center_2020_06_01_09_46_28_202.jpg
training_si

   26/13103 [..............................] - ETA: 30:03 - loss: 31504460.6921training_simulator_images/IMG/center_2020_06_01_09_35_56_804.jpg
training_simulator_images/IMG/center_2020_06_01_14_18_05_505.jpg
training_simulator_images/IMG/center_2020_06_01_14_17_18_546.jpg
training_simulator_images/IMG/center_2020_06_01_09_36_51_035.jpg
training_simulator_images/IMG/center_2020_06_01_09_34_36_129.jpg
training_simulator_images/IMG/center_2020_06_01_09_48_01_124.jpg
   27/13103 [..............................] - ETA: 30:02 - loss: 30381147.9164training_simulator_images/IMG/center_2020_06_01_13_22_07_886.jpg
training_simulator_images/IMG/center_2020_06_01_13_22_09_746.jpg
training_simulator_images/IMG/center_2020_06_01_13_25_06_009.jpg
training_simulator_images/IMG/center_2020_06_01_09_44_40_240.jpg
training_simulator_images/IMG/center_2020_06_01_14_15_34_056.jpg
training_simulator_images/IMG/center_2020_06_01_09_46_42_491.jpg
   28/13103 [..............................] - ETA: 30:02 - lo

training_simulator_images/IMG/center_2020_06_01_13_24_23_960.jpg
training_simulator_images/IMG/center_2020_06_01_13_21_18_560.jpg
   44/13103 [..............................] - ETA: 29:30 - loss: 19552401.8549training_simulator_images/IMG/center_2020_06_01_13_22_27_614.jpg
training_simulator_images/IMG/center_2020_06_01_09_46_11_280.jpg
training_simulator_images/IMG/center_2020_06_01_09_44_35_623.jpg
training_simulator_images/IMG/center_2020_06_01_09_53_56_885.jpg
training_simulator_images/IMG/center_2020_06_01_09_38_45_482.jpg
   45/13103 [..............................] - ETA: 29:39 - loss: 19119671.6437
training_simulator_images/IMG/center_2020_06_01_09_44_42_580.jpg
training_simulator_images/IMG/center_2020_06_01_09_44_30_763.jpg
training_simulator_images/IMG/center_2020_06_01_14_17_05_478.jpg
training_simulator_images/IMG/center_2020_06_01_09_36_13_684.jpg
training_simulator_images/IMG/center_2020_06_01_09_35_53_338.jpg
training_simulator_images/IMG/center_2020_06_01_09_35_34_064.

training_simulator_images/IMG/center_2020_06_01_09_34_30_631.jpg
training_simulator_images/IMG/center_2020_06_01_14_18_33_858.jpg
training_simulator_images/IMG/center_2020_06_01_13_24_02_019.jpg
training_simulator_images/IMG/center_2020_06_01_09_34_40_051.jpg
   62/13103 [..............................] - ETA: 31:21 - loss: 13978663.6291training_simulator_images/IMG/center_2020_06_01_09_37_46_094.jpg
training_simulator_images/IMG/center_2020_06_01_09_36_39_012.jpg
training_simulator_images/IMG/center_2020_06_01_14_16_53_334.jpg
training_simulator_images/IMG/center_2020_06_01_09_38_40_864.jpg
training_simulator_images/IMG/center_2020_06_01_09_34_36_265.jpg
training_simulator_images/IMG/center_2020_06_01_09_36_38_174.jpg
   63/13103 [..............................] - ETA: 31:28 - loss: 13757610.9985training_simulator_images/IMG/center_2020_06_01_13_21_14_007.jpg
training_simulator_images/IMG/center_2020_06_01_14_14_33_389.jpg
training_simulator_images/IMG/center_2020_06_01_09_46_56_340.j

   79/13103 [..............................] - ETA: 31:35 - loss: 10996987.0026training_simulator_images/IMG/center_2020_06_01_09_38_15_380.jpg
training_simulator_images/IMG/center_2020_06_01_13_23_05_820.jpg
training_simulator_images/IMG/center_2020_06_01_09_36_50_823.jpg
training_simulator_images/IMG/center_2020_06_01_14_17_29_573.jpg
training_simulator_images/IMG/center_2020_06_01_14_18_59_573.jpg
training_simulator_images/IMG/center_2020_06_01_14_15_55_065.jpg
   80/13103 [..............................] - ETA: 31:33 - loss: 10860291.6391training_simulator_images/IMG/center_2020_06_01_13_25_06_393.jpg
training_simulator_images/IMG/center_2020_06_01_13_25_04_220.jpg
training_simulator_images/IMG/center_2020_06_01_13_21_45_440.jpg
training_simulator_images/IMG/center_2020_06_01_09_44_13_425.jpg
training_simulator_images/IMG/center_2020_06_01_09_37_51_585.jpg
training_simulator_images/IMG/center_2020_06_01_13_24_28_264.jpg
   81/13103 [..............................] - ETA: 31:34 - lo

training_simulator_images/IMG/center_2020_06_01_14_17_25_967.jpg
training_simulator_images/IMG/center_2020_06_01_14_14_56_635.jpg
   97/13103 [..............................] - ETA: 32:31 - loss: 8968516.9750training_simulator_images/IMG/center_2020_06_01_09_46_23_637.jpg
training_simulator_images/IMG/center_2020_06_01_14_15_19_767.jpg
training_simulator_images/IMG/center_2020_06_01_09_54_50_773.jpg
training_simulator_images/IMG/center_2020_06_01_09_45_10_134.jpg
training_simulator_images/IMG/center_2020_06_01_13_20_48_564.j
   98/13103 [..............................] - ETA: 32:26 - loss: 8877159.6978training_simulator_images/IMG/center_2020_06_01_14_15_58_290.jpg
training_simulator_images/IMG/center_2020_06_01_09_55_01_578.jpg
training_simulator_images/IMG/center_2020_06_01_13_21_22_826.jpg
training_simulator_images/IMG/center_2020_06_01_09_46_37_782.jpg
training_simulator_images/IMG/center_2020_06_01_09_47_41_553.jpg
training_simulator_images/IMG/center_2020_06_01_09_45_05_648.jpg
 

KeyboardInterrupt: 

training_simulator_images/IMG/center_2020_06_01_09_37_01_738.jpg


# Preprocess data

In [6]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda

model = Sequential()
model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(160, 320, 3)))
model.add(Flatten())
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=2) #2 epochs

model.save('model.h5')

C:\Users\gusta\Anaconda3\envs\Project3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  # Remove the CWD from sys.path while we load stuff.


Train on 3859 samples, validate on 965 samples
Epoch 1/2
3859/3859 [==============================] - 7s 2ms/step - loss: 2.7471 - val_loss: 0.9103
Epoch 2/2
3859/3859 [==============================] - 7s 2ms/step - loss: 1.8588 - val_loss: 2.9442


# Testing training dataset against LeNet with changes

In [68]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.layers import Dropout, Flatten, Lambda, Cropping2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
import math

model = Sequential()
model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(160, 320, 3)))
model.add(Cropping2D(cropping=((70,25), (0, 0))))
#results into an image of shape 156x316x60
model.add(Conv2D(60, (5,5), activation='relu'))

#results into images of shape 152x312x60
model.add(Conv2D(60, (5,5), activation='relu'))
#pooling layers
#by applying a filter of 2x2, scales down the image to 72x156x60
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

#results into an image shape of 68x152x60
model.add(Conv2D(60, (5,5), activation='relu'))
#results into an image shape of 34x76x60
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

#results into an image shape of 30x72x30
model.add(Conv2D(30, (5,5), activation='relu'))\
#results into and image shape of 15x36x30
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

#results into an image shape of 11x32x30
#model.add(Conv2D(15, (5,5), activation='relu'))\
#results into and image shape of 6x18x15
#model.add(MaxPooling2D(pool_size=(2,2)))
#model.add(Dropout(0.5))

#6x18x15 = 720 nodes
model.add(Flatten())
#model.add(Dense(500))
model.add(Dense(120))
model.add(Dense(84))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
history_object = model.fit_generator(train_generator, 
            steps_per_epoch=math.ceil(len(train_samples)/batch_size), 
            validation_data=validation_generator, 
            validation_steps=math.ceil(len(validation_samples)/batch_size), 
            epochs=5, verbose=1, shuffle=1)

model.save('model.h5')

Epoch 1/5
19654
78614
  373/13103 [..............................] - ETA: 5:54:18 - loss: 0.0945

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

print(history_object.history.keys())

plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()

In [12]:
import keras
import tensorflow as tf

config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 8} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)